# User Behavior 데이터 전처리

이 노트북은 Object Storage(S3)에서 Parquet 파일을 읽어와서 다음 단계를 수행합니다:
1. S3 클라이언트 설정
2. Parquet 파일 읽기 및 하나의 DataFrame으로 병합
3. 기능 엔지니어링
   - 로그인 여부 플래그 생성
   - 나이(age) 세그먼트화
   - 성별(gender) 원-핫 인코딩
   - 현재 상태(current_state) 원-핫 인코딩 및 저빈도 제거
   - 수치형 컬럼 결측치 처리, 로그 변환, 스케일링
4. 로컬 CSV 저장
5. S3에 Parquet로 저장


In [ ]:
import os
import io
import boto3
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1) S3 클라이언트 설정
bucket = os.environ['S3_BUCKET']
prefix = os.environ['S3_PREFIX']
region = os.environ.get('AWS_REGION', 'kr-central-2')
endpoint = os.environ.get('S3_ENDPOINT_URL')
access = os.environ['S3_ACCESS_KEY_ID']
secret = os.environ['S3_SECRET_ACCESS_KEY']

s3 = boto3.client(
    's3', region_name=region, endpoint_url=endpoint,
    aws_access_key_id=access, aws_secret_access_key=secret
)

# 2) Parquet 읽기 및 병합
objs = s3.list_objects_v2(Bucket=bucket, Prefix=prefix).get('Contents', [])
keys = [o['Key'] for o in objs if o['Key'].endswith('.parquet')]
dfs = []
for key in keys:
    buf = io.BytesIO(s3.get_object(Bucket=bucket, Key=key)['Body'].read())
    dfs.append(pq.read_table(buf).to_pandas())
df = pd.concat(dfs, ignore_index=True)
print(f"Loaded {len(keys)} files, {len(df)} rows")

# 3) 기능 엔지니어링
df['is_logged_in'] = df['user_id'].fillna('unknown').ne('unknown').astype(int)

def age_seg(v):
    try: i = int(v)
    except: return 'unknown'
    return 'young' if i<25 else 'middle' if i<50 else 'old'
df['age_segment'] = df['age'].apply(age_seg)
df = df[df['age_segment']!='unknown']

df['gender'] = df['gender'].fillna('unknown')
df = pd.get_dummies(df, columns=['gender'], prefix='gender')

# 상태 원-핫 및 저빈도 제거
if 'current_state' in df:
    df = pd.get_dummies(df, columns=['current_state'], prefix='state')
    cols = [c for c in df if c.startswith('state_')]
    freq = df[cols].sum()
    drop = freq[freq<=0.01*len(df)].index
    df.drop(columns=drop, inplace=True)

# 수치형 처리
num = df.select_dtypes(include=[np.number]).columns
df[num] = df[num].fillna(0)
for c in ['last_action_elapsed','avg_response_time','session_duration']:
    if c in df: df[c]=np.log1p(df[c])
sc_cols = [c for c in ['search_count','cart_item_count','page_depth'] if c in df]
if sc_cols:
    df[sc_cols] = StandardScaler().fit_transform(df[sc_cols])

# 4) 로컬 CSV 저장
df.to_csv('processed_user_behavior.csv', index=False)

# 5) S3 Parquet 저장
out_pref = os.environ.get('PREPROCESS_PREFIX','preprocessed/')
buf = io.BytesIO()
df.to_parquet(buf, engine='pyarrow', index=False)
buf.seek(0)
s3.put_object(Bucket=bucket, Key=prefix+out_pref+'processed_user_behavior.parquet', Body=buf.getvalue())
